## train.csv의 output을 훈련 데이터로 사용한 파인 튜닝 모델을 통해 리뷰 생성

In [1]:
import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

In [2]:
BASE_MODEL = "beomi/gemma-ko-7b"
FINETUNE_MODEL = "./gemma-ko-7b-finetuning-generate-reviews"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
pipe_finetuned = pipeline(task="text-generation", model=finetune_model, tokenizer=tokenizer)

Device set to use cuda:0


In [4]:
def remove_incomplete_sentence(text):
    # 마지막 마침표(.) 또는 느낌표(!)의 위치 찾기
    last_period_index = text.rfind('.')
    last_exclamation_index = text.rfind('!')
    
    # 두 개 중 더 뒤에 있는 위치 찾기
    last_index = max(last_period_index, last_exclamation_index)

    if last_index != -1:
        return text[:last_index + 1]
    else:
        # 마침표(.)나 느낌표(!)가 없으면 None 반환
        return None

In [5]:
restored_reviews = []

prompt = r"""<bos><start_of_turn>user
이 숙박시설에 대한 솔직한 리뷰를 작성해 주세요:
<end_of_turn>
<start_of_turn>model
"""

prompts = [prompt] * 100  # 프롬프트 n개를 리스트로 생성

# 병렬 처리
generated = pipe_finetuned( 
    prompts,
    num_return_sequences=1,
    temperature=1.0,
    top_p=0.9,
    max_new_tokens=512,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    batch_size=8  # 병렬 처리 크기
)

# 모든 생성된 텍스트 처리
for gen in generated:
    generated_text = gen[0]['generated_text']
    result = generated_text[len(prompt):].strip()
    result = remove_incomplete_sentence(result)  # 문장 정리

    if result is not None:
        # <end_of_turn> 기준으로 모델 응답만 추출
        result = result.split('<end_of_turn>\n<start_of_turn>model\n')
        restored_reviews.extend(result)  # 리스트에 추가

In [6]:
df = pd.DataFrame({'input': ["" for _ in range(len(restored_reviews))],
                   'output': restored_reviews})

df.to_csv('./data/generated_reviews.csv', index = False, encoding = 'utf-8-sig')